In [1]:
import pandas as pd
import numpy as np
import webbrowser
from urllib.request import urlopen

### Functions

In [2]:
def load_p_line_cruise(url):
    txt = urlopen(url).read()
    decode_txt = txt.decode('windows-1252')
    raw_csv = decode_txt.split('\r\n')

    count = 0
    for line in raw_csv:
        if '.ctd' not in line:
            count = count+1
        else:
            break

    header = raw_csv[count-2].split(',')
    clean_csv =raw_csv[count:]

    data = []
    for row in clean_csv:
        data.append(row.split(','))

    df = pd.DataFrame(data,columns=header)

    # Get P-Line Stations only
    df.dropna(subset=['Pressure:CTD [dbar]'], inplace=True)
    df = df[df['LOC:STATION'].str.startswith('P')]
    df = df.apply(pd.to_numeric, errors='ignore')

    df.sort_values(by=['LOC:STATION','Pressure:CTD [dbar]'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df


def get_station_26(df):
    try:
        df = df[df['station'] == 'P26']
    except:
        df = df[df['LOC:STATION'] == 'P26']
    df.reset_index(drop=True,inplace=True)
    return df

def get_station_1(df):
    try:
        df = df[df['station'] == 'P1']
    except:
        df = df[df['LOC:STATION'] == 'P1']
    df.reset_index(drop=True,inplace=True)
    return df

def get_station(df,station):
    stn = 'P' + station
    try:
        df = df[df['station'] == stn]
    except:
        df = df[df['LOC:STATION'] == stn]
    df.reset_index(drop=True,inplace=True)
    return df

def rename_pline_columns(df):
    
    df.rename(columns={'LOC:LATITUDE':'latitude', 'LOC:LONGITUDE':'longitude','LOC:STATION':'station', 'Pressure:CTD [dbar]':'sea_water_pressure','Temperature:CTD [deg_C_(ITS90)]':'sea_water_temperature','Oxygen:Dissolved:CTD:Mass [µmol/kg]':'moles_of_oxygen_per_unit_mass_in_sea_water','Fluorescence:CTD:Seapoint [mg/m^3]':'mass_concentration_of_chlorophyll'},inplace=True)
    return df

def build_url(year,month):
    url = 'https://www.waterproperties.ca/linep/' + year + '-' + month + '/donneesctddata/' + year + '-' + month + '-ctd-cruise.csv'
    return url

### Cruise Lists

## 2009 01 cruise file was uploaded incorrectly and has an extra STATION column rename or delete this column

In [3]:
cruise_list =[ 
    ('2018','001'),
    ('2017', '01'), ('2017','06'), ('2017','08'),
    ('2016', '01'), ('2016','06'), ('2016','08'),
    ('2015', '01'), ('2015', '09'), ('2015', '10'),
    ('2014', '01'), ('2014', '18'), ('2014', '19'),
    ('2013', '01'), ('2013', '17'), ('2013', '18'),
    ('2012', '01'), ('2012', '12'), ('2012', '13'),
    ('2011', '01'), ('2011', '26'), ('2011', '27'),
    ('2010', '01'), ('2010', '13'), ('2010', '14'),
    ('2009', '09'), ('2009', '10'), #2009 03 is omitted due to formatting error
    ('2008', '01'), ('2008', '26'), ('2008', '27'),
    ('2007', '01'), ('2007', '13'), ('2007', '15'),
]

In [4]:
"""
2018 001
2017 01 06 08
2016 01 06 08
2015 01 09 10
2014 01 18 19
2013 01 17 18
2012 01 12 13
2011 01 26 27
2010 01 13 14
2009 03 09 10
2008 01 26 27
2007 01 13 15

"""

'\n2018 001\n2017 01 06 08\n2016 01 06 08\n2015 01 09 10\n2014 01 18 19\n2013 01 17 18\n2012 01 12 13\n2011 01 26 27\n2010 01 13 14\n2009 03 09 10\n2008 01 26 27\n2007 01 13 15\n\n'

### Run Code With Saving

In [29]:
for cruise in cruise_list:
    url = build_url(cruise[0],cruise[1])
    df = load_p_line_cruise(url)
    df = rename_pline_columns(df)
    df_1 = get_station_1(df)
    df_26 = get_station_26(df)
    
    df1_file = './line_p/cruise-' +cruise[0] + '-' + cruise[1] +'-station-1.csv'
    df26_file = './line_p/cruise-' +cruise[0] + '-' + cruise[1] +'-station-26.csv'
    df_1.to_csv(df1_file,index=False)
    df_26.to_csv(df26_file,index=False)
    
    

### GRAVEYARD

### Individual Case

In [6]:
url = build_url('2010','01')
df = load_p_line_cruise(url)
df = rename_pline_columns(df) #
df_1 = get_station_1(df)
df_26 = get_station_26(df)


### 2009 Special Case

In [28]:
url = build_url('2009','03')
df = load_p_line_cruise(url)
df = rename_pline_columns(df) #
df_1 = get_station_1(df)
df_26 = get_station_26(df)

df1_file = './line_p/cruise-' +'2009' + '-' + '03' +'-station-1.csv'
df26_file = './line_p/cruise-' +'2009' + '-' + '03' +'-station-26.csv'

df_1.to_csv(df1_file)
df_26.to_csv(df26_file)

In [ ]:
### Urls:
 line_2018_01 = 'https://www.waterproperties.ca/linep/2018-001/donneesctddata/2018-001-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2017-01/donneesctddata/2017-01-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2017-06/donneesctddata/2017-06-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2017-08/donneesctddata/2017-08-ctd-cruise.csv'

'https://www.waterproperties.ca/linep/2016-01/donneesctddata/2016-01-ctd-cruise.csv'

'https://www.waterproperties.ca/linep/2009-03/donneesctddata/2009-03-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2009-09/donneesctddata/2009-09-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2009-10/donneesctddata/2009-10-ctd-cruise.csv'

'https://www.waterproperties.ca/linep/2008-01/donneesctddata/2008-01-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2008-26/donneesctddata/2008-26-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2008-27/donneesctddata/2008-27-ctd-cruise.csv'

'https://www.waterproperties.ca/linep/2007-01/donneesctddata/2007-01-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2007-01/donneesctddata/2007-01-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2007-13/donneesctddata/2007-13-ctd-cruise.csv'
'https://www.waterproperties.ca/linep/2007-15/donneesctddata/2007-15-ctd-cruise.csv'
